### imports

In [1]:
import os
import pandas as pd

# widget for convenient view of dataframes
import qgrid
from qgrid import show_grid as sg
qgrid.set_grid_option('forceFitColumns', False)
qgrid.set_grid_option('defaultColumnWidth', 180)
qgrid.set_defaults(show_toolbar=True)

### read data from `.xlsx`

In [2]:
colnames_map = {
    'Раён': 'district',
    'Гайдукевіч, N': 'H_n',
    'Гайдукевіч, %': 'H_%',
    'Караткевіч, N': 'K_n',
    'Караткевіч, %': 'K_%',
    'Лукашэнка, N': 'L_n',
    'Лукашэнка, %': 'L_%',
    'Улаховіч, N': 'U_n',
    'Улаховіч, %': 'U_%',
    'выбарцы агулам': 'electors',
    'новыя выбарцы': 'new_electors',
    'атрыманыя бюлетэні': 'ballots',
    'прынялі, N': 'voted_n',
    'прынялі, %': 'voted_%',
    'датэрмінова': 'early_voters',
    'па месцы знаходжання': 'home_voters',
    'на ўчастку': 'pp_voters',  # polling place voters
    'супраць усіх': 'against_all',
    'несапраўдныя бюлетэні': 'spoilt',
}

In [3]:
fp = os.path.join('..', 'data', 'original', 'Vybary_2015_rajonnaja_statystyka.xlsx')

region_sheets = [
    'Miensk',
    'Mienskaja_voblasc', 'Bresckaja_voblasc', 'Haradzienskaja_voblasc',
    'Viciebskaja_voblasc', 'Mahiliouskaja_voblasc', 'Homielskaja_voblasc',
    'Respublika'
]

dfs = []
for region in region_sheets:
    cur_df = pd.read_excel(fp, sheet_name=region, skiprows=9, na_values='-')
    cur_df.rename(colnames_map, axis=1, inplace=True)
    cur_df['region'] = region
    dfs.append(cur_df)
df = pd.concat(dfs, axis=0, ignore_index=True)
df.shape

(161, 20)

* map region names

In [4]:
regions_map = {
    'Miensk': 'г. Менск',
    'Mienskaja_voblasc': 'Менская вобласць',
    'Bresckaja_voblasc': 'Брэсцкая вобласць',
    'Haradzienskaja_voblasc': 'Гарадзенская вобласць',
    'Viciebskaja_voblasc': 'Віцебская вобласць',
    'Mahiliouskaja_voblasc': 'Магілёўская вобласць',
    'Homielskaja_voblasc': 'Гомельская вобласць',
    'Respublika': 'Рэспубліка'
}
df['region'] = df['region'].map(regions_map)

* add `district_type` column

In [5]:
df['district_type'] = 'раён'
df.loc[df['district'].str.contains('г\.'), 'district_type'] = 'раён гораду'
df.loc[df['district'].str.contains('^г\.'), 'district_type'] = 'горад'
df.loc[df['region'] == 'г. Менск', 'district_type'] = 'раён гораду'

df.loc[df['district'] == 'Агулам', 'district_type'] = 'Агулам'
df.loc[df['region'] == 'Рэспубліка', 'district_type'] = 'Агулам'

* move some columns to the beginning

In [6]:
def pullup(df, cols_to_pull):
    """Move some columns to the beginning of dataframe."""
    if isinstance(cols_to_pull, str):
        cols_to_pull = [cols_to_pull]
    cols_to_pull = list(cols_to_pull)
    remaining_cols = [col for col in df.columns if col not in set(cols_to_pull)]
    res = df[cols_to_pull + remaining_cols]
    return res    

In [7]:
df = pullup(df, ['region', 'district', 'district_type'])

* Make `district` columns explicit

In [8]:
ix = (df['district'] == 'Агулам')
df.loc[ix, 'district'] = df[ix]['region'] + '. ' + df[ix]['district']

ix = (df['region'] == 'Рэспубліка') & ((df['district'] != 'Рэспубліка. Агулам'))
df.loc[ix, 'district'] = df[ix]['district'] + '. ' + df[ix]['district_type']

* show dataframe

In [9]:
df

,region,district,district_type,H_n,H_%,K_n,K_%,L_n,L_%,U_n,...,electors,new_electors,ballots,voted_n,voted_%,early_voters,home_voters,pp_voters,against_all,spoilt
0,г. Менск,Завадскі,раён гораду,4428,3.64,9221,7.58,75931,62.42,3966,...,166138,465.0,121646,121639,73.22,48601,837,72201,27414,679.0
1,г. Менск,Ленінскі,раён гораду,4068,3.37,9952,8.24,77753,64.34,3451,...,165567,713.0,120995,120844,72.99,43996,2796,74052,25263,357.0
2,г. Менск,Маскоўскі,раён гораду,4753,3.21,9472,6.40,99879,67.50,3731,...,198655,376.0,148082,147972,74.49,60939,1791,85242,29289,848.0
3,г. Менск,Кастрычніцкі,раён гораду,3077,3.36,6569,7.17,61000,66.62,2365,...,124966,5.0,91631,91566,73.27,33855,962,56749,18165,390.0
4,г. Менск,Партызанскі,раён гораду,1870,3.79,3868,7.83,30542,61.85,1598,...,67502,109.0,49385,49379,73.15,19231,2164,27984,10914,587.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,Рэспубліка,Гарадзенская вобласць. Агулам,Агулам,15691,2.27,32755,4.75,592490,85.89,7092,...,779638,2105.0,689841,689824,88.48,286439,64395,338990,34169,7627.0
157,Рэспубліка,Менская вобласць. Агулам,Агулам,32128,3.21,44155,4.41,858287,85.74,21763,...,1116445,2019.0,1001107,1001041,89.66,412680,97707,490654,37552,7156.0
158,Рэспубліка,Магілёўская вобласць. Агулам,Агулам,27173,3.66,25652,3.46,655396,88.30,7928,...,816030,1420.0,742575,742263,90.96,298268,59198,384797,19443,6671.0
159,Рэспубліка,г. Менск. Агулам,Агулам,32585,3.33,69134,7.07,642119,65.69,26628,...,1314164,3807.0,978800,977448,74.38,384285,20561,572602,199056,7926.0


In [10]:
sg(df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

### check NANs and dtypes

In [11]:
df.isna().sum().to_frame('na_count').query('na_count > 0')

,na_count
new_electors,42
spoilt,2


In [12]:
df[df['spoilt'].isna()]

,region,district,district_type,H_n,H_%,K_n,K_%,L_n,L_%,U_n,...,electors,new_electors,ballots,voted_n,voted_%,early_voters,home_voters,pp_voters,against_all,spoilt
48,Брэсцкая вобласць,Пружанскі,раён,457,1.25,1764,4.82,33871,92.62,243,...,37801,166.0,36569,36569,96.74,14654,4268,17647,234,NaN
96,Віцебская вобласць,г. Наваполацак,горад,3970,5.90,2207,3.28,55576,82.62,2085,...,74676,334.0,67269,67269,90.08,30263,2300,34706,3431,NaN


In [13]:
df[df['new_electors'].isna()]

,region,district,district_type,H_n,H_%,K_n,K_%,L_n,L_%,U_n,...,electors,new_electors,ballots,voted_n,voted_%,early_voters,home_voters,pp_voters,against_all,spoilt
5,г. Менск,Першамайскі,раён гораду,3873,3.20,8549,7.07,80881,66.89,3212,...,158455,NaN,121004,120919,76.31,47962,4469,68488,23092,1312.0
21,Менская вобласць,Маладзечанскі,раён,2325,2.44,5531,5.81,77384,81.28,1360,...,106959,NaN,95208,95208,89.01,39362,7045,48801,6927,1681.0
22,Менская вобласць,Мядзельскі,раён,871,4.26,531,2.60,17798,86.99,589,...,22238,NaN,20459,20459,92.00,8242,3730,8487,654,16.0
26,Менская вобласць,Смалявіцкі,раён,618,1.89,1266,3.88,28426,87.10,483,...,35333,NaN,32637,32637,92.37,11924,4268,16445,1115,729.0
31,Менская вобласць,Чэрвеньскі,раён,891,3.73,593,2.49,21437,89.85,206,...,24979,NaN,23859,23859,95.52,10113,3242,10504,537,195.0
55,Гарадзенская вобласць,Берастовіцкі,раён,167,1.42,208,1.77,10960,93.05,104,...,12016,NaN,11779,11779,98.03,5437,1612,4730,317,23.0
60,Гарадзенская вобласць,Зельвенскі,раён,192,1.49,310,2.41,11966,93.08,76,...,13194,NaN,12856,12856,97.44,6028,2220,4608,247,65.0
61,Гарадзенская вобласць,Іўеўскі,раён,277,1.38,336,1.67,18734,93.01,101,...,20755,NaN,20141,20141,97.04,8373,1950,9818,263,430.0
62,Гарадзенская вобласць,Карэліцкі,раён,108,0.64,501,2.98,15946,94.89,81,...,17455,NaN,16805,16805,96.28,7496,3970,5339,142,27.0
66,Гарадзенская вобласць,Астравецкі,раён,264,1.43,328,1.77,17204,93.05,115,...,19458,NaN,18489,18489,95.02,8023,2965,7501,463,115.0


* Fill missed values with 0

In [14]:
df = df.fillna(0, downcast='infer')

In [15]:
df.dtypes

region            object
district          object
district_type     object
H_n                int64
H_%              float64
K_n                int64
K_%              float64
L_n                int64
L_%              float64
U_n                int64
U_%              float64
electors           int64
new_electors       int64
ballots            int64
voted_n            int64
voted_%          float64
early_voters       int64
home_voters        int64
pp_voters          int64
against_all        int64
spoilt             int64
dtype: object

### save the dataset to `.csv` and `.xlsx`

In [16]:
out_dp = os.path.join('..', 'data', 'output')
os.makedirs(out_dp, exist_ok=True)

In [17]:
out_fp_csv = os.path.join(out_dp, 'Vybary_2015.csv')
df.to_csv(out_fp_csv, index=False)

In [18]:
out_fp_xlsx = os.path.join(out_dp, 'Vybary_2015.xlsx')
df.to_excel(out_fp_xlsx, index=False, freeze_panes=(1,0))